<div class="alert alert-block alert-success">
    <h1>
        Example notebook - Healthcare
    </h1>
    <p>
        Link to dataset : <a href="https://www.kaggle.com/datasets/prasad22/healthcare-dataset">Kaggle link</a>
    </p>
</div>

# Import modules and functions

In [1]:
import os
import pandas as pd
import re

from turingdb_examples.utils import create_ID_column, get_return_statements
from turingdb_examples.graph import (
    create_graph_from_df,
    build_create_command_from_networkx,
)
from turingdb_examples.llm import natural_language_to_cypher

# Check data files are available

In [2]:
example_name = "healthcare_dataset"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    raise ValueError(f"{path_data} does not exists")

list_csv_files = sorted(os.listdir(path_data))
if not list_csv_files == ["healthcare_dataset.csv"]:
    raise ValueError(f"csv file is not available in {path_data}")

# Import and format data

In [3]:
df = pd.read_csv(f"{path_data}/healthcare_dataset.csv")
df["Name"] = df["Name"].apply(
    lambda x: f"{x.split(' ')[0].capitalize()} {x.split(' ')[1].upper()}"
)
df["Doctor"] = df["Doctor"].apply(
    lambda x: f"{x.split(' ')[0].capitalize()} {x.split(' ')[1].upper()}"
)
df = create_ID_column(df)
# Keep only 10 patients to reduce graph for now
# You can comment the following line to generate the whole graph
df = df.iloc[:10, :]
df

,Patient ID,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,00000,Bobby JACKSON,30,Male,B-,Cancer,2024-01-31,Matthew SMITH,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,00001,Leslie TERRY,62,Male,A+,Obesity,2019-08-20,Samantha DAVIES,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,00002,Danny SMITH,76,Female,A-,Obesity,2022-09-22,Tiffany MITCHELL,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,00003,Andrew WATTS,28,Female,O+,Diabetes,2020-11-18,Kevin WELLS,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,00004,Adrienne BELL,43,Female,AB+,Cancer,2022-09-19,Kathleen HANNA,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal
5,00005,Emily JOHNSON,36,Male,A+,Asthma,2023-12-20,Taylor NEWTON,Nunez-Humphrey,UnitedHealthcare,48145.110951,389,Urgent,2023-12-24,Ibuprofen,Normal
6,00006,Edward EDWARDS,21,Female,AB-,Diabetes,2020-11-03,Kelly OLSON,Group Middleton,Medicare,19580.872345,389,Emergency,2020-11-15,Paracetamol,Inconclusive
7,00007,Christina MARTINEZ,20,Female,A+,Cancer,2021-12-28,Suzanne THOMAS,"Powell Robinson and Valdez,",Cigna,45820.462722,277,Emergency,2022-01-07,Paracetamol,Inconclusive
8,00008,Jasmine AGUILAR,82,Male,AB+,Asthma,2020-07-01,Daniel FERGUSON,Sons Rich and,Cigna,50119.222792,316,Elective,2020-07-14,Aspirin,Abnormal
9,00009,Christopher BERG,58,Female,AB-,Cancer,2021-05-23,Heather DAY,Padilla-Walker,UnitedHealthcare,19784.631062,249,Elective,2021-06-22,Paracetamol,Inconclusive


# Create graph from dataframe

In [4]:
label_str = "displayName"

G = create_graph_from_df(
    df,
    directed=True,
    source_node_col={"id": "Patient ID", label_str: "Name", "type": "Patient"},
    attributes_source_node_cols=["Age", "Date of Admission", "Discharge Date"],
    optional_nodes_cols={
        "Gender": {"link_to_source": True, "edge_type_to_source": "is"},
        "Blood Type": {"link_to_source": True, "edge_type_to_source": "is"},
        "Medical Condition": {"link_to_source": True, "edge_type_to_source": "has"},
        "Doctor": {"link_to_source": True, "edge_type_to_source": "is_treated_by"},
        "Hospital": {
            "attributes": ["Room Number"],
            "link_to_source": True,
            "edge_type_to_source": "is_treated_in",
        },
        "Insurance Provider": {
            "attributes": ["Billing Amount"],
            "link_to_source": True,
            "edge_type_to_source": "is_client_of",
        },
        "Admission Type": {"link_to_source": True},
        "Medication": {
            "link_to_source": True,
            "edge_type_to_source": "took_medication",
        },
        "Test Results": {"link_to_source": True, "edge_type_to_source": "has_result"},
    },
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 57 nodes and 90 edges


In [5]:
# Show first few nodes with properties
for node in list(G.nodes(data=True))[:20]:
    print(node)

('00000', {'displayName': 'Bobby JACKSON', 'type': 'Patient', 'Age': 30, 'Date of Admission': '2024-01-31', 'Discharge Date': '2024-02-02'})
('Male', {'displayName': 'Male', 'type': 'Gender'})
('B-', {'displayName': 'B-', 'type': 'Blood Type'})
('Cancer', {'displayName': 'Cancer', 'type': 'Medical Condition'})
('Matthew SMITH', {'displayName': 'Matthew SMITH', 'type': 'Doctor'})
('Sons and Miller', {'displayName': 'Sons and Miller', 'type': 'Hospital', 'Room Number': 328})
('Blue Cross', {'displayName': 'Blue Cross', 'type': 'Insurance Provider', 'Billing Amount': 18856.281305978155})
('Urgent', {'displayName': 'Urgent', 'type': 'Admission Type'})
('Paracetamol', {'displayName': 'Paracetamol', 'type': 'Medication'})
('Normal', {'displayName': 'Normal', 'type': 'Test Results'})
('00001', {'displayName': 'Leslie TERRY', 'type': 'Patient', 'Age': 62, 'Date of Admission': '2019-08-20', 'Discharge Date': '2019-08-26'})
('A+', {'displayName': 'A+', 'type': 'Blood Type'})
('Obesity', {'displa

In [6]:
# Show first few edge with properties
for edge in list(G.edges(data=True))[:20]:
    print(edge)

('00000', 'Male', {'type': 'is'})
('00000', 'B-', {'type': 'is'})
('00000', 'Cancer', {'type': 'has'})
('00000', 'Matthew SMITH', {'type': 'is_treated_by'})
('00000', 'Sons and Miller', {'type': 'is_treated_in'})
('00000', 'Blue Cross', {'type': 'is_client_of'})
('00000', 'Urgent', {})
('00000', 'Paracetamol', {'type': 'took_medication'})
('00000', 'Normal', {'type': 'has_result'})
('00001', 'Male', {'type': 'is'})
('00001', 'A+', {'type': 'is'})
('00001', 'Obesity', {'type': 'has'})
('00001', 'Samantha DAVIES', {'type': 'is_treated_by'})
('00001', 'Kim Inc', {'type': 'is_treated_in'})
('00001', 'Medicare', {'type': 'is_client_of'})
('00001', 'Emergency', {})
('00001', 'Ibuprofen', {'type': 'took_medication'})
('00001', 'Inconclusive', {'type': 'has_result'})
('00002', 'Female', {'type': 'is'})
('00002', 'A-', {'type': 'is'})


# Create graph using `turingdb` python package

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        <ul>
            <li>Create your TuringDB account</li>
            <li>Create your instance in the <a href="https://console.turingdb.ai/auth">TuringDB Cloud UI</a></li>
            <li>Copy your Instance ID from the Database Instances management page</li>
            <li>Get API Key from the Settings in UI</li>
        </ul>
        Remember to have your instance active while working in this notebook !
    </h3>
</div>

In [7]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(
    host="http://localhost:6666"  # Remove this parameter and set the two parameters below
    # instance_id="...",  # Replace by your instance id
    # auth_token="...",  # Replace by your API token
)

In [8]:
# Get list of available graphs
list_graphs = client.query("LIST GRAPH").loc[:, 0].tolist()

In [9]:
# Set graph name
graph_name_prefix = example_name
graph_name_nb_suffix = str(
    max(
        [
            int(re.sub(graph_name_prefix, "", g))
            for g in list_graphs
            if g.startswith(graph_name_prefix)
            and re.sub(graph_name_prefix, "", g).isdigit()
        ]
        + [0]
    )
    + 1
)
graph_name = graph_name_prefix + graph_name_nb_suffix
graph_name = re.sub("-", "_", graph_name)
graph_name

'healthcare_dataset2'

In [10]:
%%time

# Create a new graph
client.query(f"CREATE GRAPH {graph_name}")
client.set_graph(graph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW").loc[0, 0]

# Checkout into the change
client.checkout(change=change)

CPU times: user 1.64 ms, sys: 976 μs, total: 2.61 ms
Wall time: 2.02 ms


In [11]:
# Build CREATE command from networkx object
create_command = build_create_command_from_networkx(G)
print(f"Cypher CREATE command :\n\n{100 * '*'}\n{create_command}\n{100 * '*'}")

Cypher CREATE command :

****************************************************************************************************
CREATE (n0:Patient {"id":"00000", "displayName":"Bobby JACKSON", "type":"Patient", "Age":"30", "Date of Admission":"2024-01-31", "Discharge Date":"2024-02-02"}),
(n1:Gender {"id":"Male", "displayName":"Male", "type":"Gender"}),
(n2:BloodType {"id":"B-", "displayName":"B-", "type":"Blood Type"}),
(n3:MedicalCondition {"id":"Cancer", "displayName":"Cancer", "type":"Medical Condition"}),
(n4:Doctor {"id":"Matthew SMITH", "displayName":"Matthew SMITH", "type":"Doctor"}),
(n5:Hospital {"id":"Sons and Miller", "displayName":"Sons and Miller", "type":"Hospital", "Room Number":"328"}),
(n6:InsuranceProvider {"id":"Blue Cross", "displayName":"Blue Cross", "type":"Insurance Provider", "Billing Amount":"18856.281305978155"}),
(n7:AdmissionType {"id":"Urgent", "displayName":"Urgent", "type":"Admission Type"}),
(n8:Medication {"id":"Paracetamol", "displayName":"Paracetamol",

In [12]:
%%time

# Run CREATE command
client.query(create_command)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

# Checkout into main
client.checkout()

CPU times: user 2.36 ms, sys: 957 μs, total: 3.32 ms
Wall time: 3.48 ms


<div class="alert alert-block alert-info">
    <h2>
        Visualize your graph in TuringDB Graph Visualizer ! Now that your instance is running:
    </h2>
    <h3>
        <ul>
            <li>Go to <a href="https://console.turingdb.ai/databases">TuringDB Console - Database Instances</a></li>
            <li>In your current instance panel, click on "Open Visualizer" button</li>
            <li>Visualizer opens, now you can choose your graph in the dropdown menu at the top-right corner</li>
        </ul>
        You can then play with your graph and visualize the nodes you want !
    </h3>
</div>

# Query TuringDB

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [13]:
%%time

# CALL LABELS () - returns a column of all the different node labels
command = """
CALL LABELS()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Node_type_ID", "Node_type"]
    display(df)

,Node_type_ID,Node_type
0,0,Patient
1,1,Gender
2,2,BloodType
3,3,MedicalCondition
4,4,Doctor
5,5,Hospital
6,6,InsuranceProvider
7,7,AdmissionType
8,8,Medication
9,9,TestResults


CPU times: user 4.91 ms, sys: 0 ns, total: 4.91 ms
Wall time: 4.26 ms


In [14]:
%%time

# CALL EDGETYPES() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL EDGETYPES()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Edge_type_ID", "Edge_type"]
    display(df)

,Edge_type_ID,Edge_type
0,0,IS
1,1,HAS
2,2,IS_TREATED_BY
3,3,IS_TREATED_IN
4,4,IS_CLIENT_OF
5,5,CONNECTED
6,6,TOOK_MEDICATION
7,7,HAS_RESULT


CPU times: user 4.4 ms, sys: 0 ns, total: 4.4 ms
Wall time: 3.76 ms


## Simple queries

In [15]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]-(m)
RETURN n.displayName, e, m.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,n.displayName,e,m.displayName
0,Danny SMITH,0,Normal
1,Danny SMITH,1,Aspirin
2,Danny SMITH,2,Emergency
3,Danny SMITH,3,Aetna
4,Danny SMITH,4,Cook PLC
...,...,...,...
85,Bobby JACKSON,85,Sons and Miller
86,Bobby JACKSON,86,Matthew SMITH
87,Bobby JACKSON,87,Cancer
88,Bobby JACKSON,88,B-


CPU times: user 4.28 ms, sys: 955 μs, total: 5.23 ms
Wall time: 4.6 ms


In [16]:
%%time

# Match all edges linking a Patient to an other node
# Return displayName and type properties
command = """
MATCH (n:Patient)-[e]-(m)
RETURN n.type, n.displayName, e, m.type, m.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,n.type,n.displayName,e,m.type,m.displayName
0,Patient,Danny SMITH,0,Test Results,Normal
1,Patient,Danny SMITH,1,Medication,Aspirin
2,Patient,Danny SMITH,2,Admission Type,Emergency
3,Patient,Danny SMITH,3,Insurance Provider,Aetna
4,Patient,Danny SMITH,4,Hospital,Cook PLC
...,...,...,...,...,...
85,Patient,Bobby JACKSON,85,Hospital,Sons and Miller
86,Patient,Bobby JACKSON,86,Doctor,Matthew SMITH
87,Patient,Bobby JACKSON,87,Medical Condition,Cancer
88,Patient,Bobby JACKSON,88,Blood Type,B-


CPU times: user 5.17 ms, sys: 0 ns, total: 5.17 ms
Wall time: 4.46 ms


In [17]:
%%time

# Find all patients
command = """
MATCH (p:Patient)
RETURN p.id, p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.id,p.displayName,p.Age
0,00002,Danny SMITH,76
1,00003,Andrew WATTS,28
2,00004,Adrienne BELL,43
3,00001,Leslie TERRY,62
4,00005,Emily JOHNSON,36
5,00006,Edward EDWARDS,21
6,00007,Christina MARTINEZ,20
7,00008,Jasmine AGUILAR,82
8,00009,Christopher BERG,58
9,00000,Bobby JACKSON,30


CPU times: user 4.44 ms, sys: 35 μs, total: 4.48 ms
Wall time: 3.82 ms


In [18]:
%%time

# Find all doctors
command = """
MATCH (d:Doctor)
RETURN d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,d.displayName
0,Kathleen HANNA
1,Samantha DAVIES
2,Heather DAY
3,Taylor NEWTON
4,Kevin WELLS
5,Kelly OLSON
6,Suzanne THOMAS
7,Matthew SMITH
8,Tiffany MITCHELL
9,Daniel FERGUSON


CPU times: user 3.48 ms, sys: 965 μs, total: 4.44 ms
Wall time: 3.81 ms


In [19]:
%%time

# Find all medications
command = """
MATCH (d:Medication)
RETURN d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,d.displayName
0,Paracetamol
1,Aspirin
2,Ibuprofen
3,Penicillin


CPU times: user 4.06 ms, sys: 73 μs, total: 4.13 ms
Wall time: 3.57 ms


In [20]:
%%time

# Find patient with specific ID and return all their information
command = """
MATCH (p:Patient {id: "00000"})
RETURN p, p.id, p.displayName, p.type, p.Age, p."Date of Admission", p."Discharge Date"
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p,p.id,p.displayName,p.type,p.Age,"p.""Date of Admission""","p.""Discharge Date"""
0,9,00000,Bobby JACKSON,Patient,30,2024-01-31,2024-02-02


CPU times: user 4.49 ms, sys: 1.04 ms, total: 5.53 ms
Wall time: 4.78 ms


In [21]:
%%time

# Find female patients
command = """
MATCH (p:Patient)-[:IS]-(g:Gender {displayName: "Female"})
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.displayName,p.Age
0,Danny SMITH,76
1,Andrew WATTS,28
2,Adrienne BELL,43
3,Edward EDWARDS,21
4,Christina MARTINEZ,20
5,Christopher BERG,58


CPU times: user 3.12 ms, sys: 940 μs, total: 4.06 ms
Wall time: 3.52 ms


In [22]:
%%time

# Find patients with Cancer
command = """
MATCH (p:Patient)-[:HAS]-(mc:MedicalCondition {displayName: "Cancer"})
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.displayName,p.Age
0,Adrienne BELL,43
1,Christina MARTINEZ,20
2,Christopher BERG,58
3,Bobby JACKSON,30


CPU times: user 3.53 ms, sys: 968 μs, total: 4.49 ms
Wall time: 3.89 ms


In [23]:
%%time

# Find all patients who are treated by a doctor
command = """
MATCH (p:Patient)-[:IS_TREATED_BY]-(d:Doctor)
RETURN p.displayName, d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.displayName,d.displayName
0,Danny SMITH,Tiffany MITCHELL
1,Andrew WATTS,Kevin WELLS
2,Adrienne BELL,Kathleen HANNA
3,Leslie TERRY,Samantha DAVIES
4,Emily JOHNSON,Taylor NEWTON
5,Edward EDWARDS,Kelly OLSON
6,Christina MARTINEZ,Suzanne THOMAS
7,Jasmine AGUILAR,Daniel FERGUSON
8,Christopher BERG,Heather DAY
9,Bobby JACKSON,Matthew SMITH


CPU times: user 3.02 ms, sys: 2.01 ms, total: 5.02 ms
Wall time: 4.35 ms


In [24]:
%%time

# Find all patients treated by doctor Kelly OLSON
command = """
MATCH (p:Patient)-[:IS_TREATED_BY]-(d:Doctor {"displayName": "Kelly OLSON"})
RETURN p.displayName, d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.displayName,d.displayName
0,Edward EDWARDS,Kelly OLSON


CPU times: user 4.09 ms, sys: 996 μs, total: 5.09 ms
Wall time: 4.05 ms


In [25]:
%%time

# Find all patients with blood type A+
command = """
MATCH (p:Patient)-[:IS]-(bt:BloodType {displayName: "A+"})
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,p.displayName,p.Age
0,Leslie TERRY,62
1,Emily JOHNSON,36
2,Christina MARTINEZ,20


CPU times: user 4.63 ms, sys: 0 ns, total: 4.63 ms
Wall time: 4.05 ms


In [26]:
%%time

# Find all patients who took Paracetamol
command = """
MATCH (p:Patient)-[:TOOK_MEDICATION]-(m:Medication {"displayName": "Paracetamol"})
RETURN p.id, p.displayName, m.displayName
"""
df = client.query(command)
df.columns = ["Patient ID", "Patient Name", "Medication"]
display(df)

,Patient ID,Patient Name,Medication
0,00006,Edward EDWARDS,Paracetamol
1,00007,Christina MARTINEZ,Paracetamol
2,00009,Christopher BERG,Paracetamol
3,00000,Bobby JACKSON,Paracetamol


CPU times: user 3.68 ms, sys: 1.05 ms, total: 4.73 ms
Wall time: 4.15 ms


# Create subgraph to visualise

In [27]:
import numpy as np

In [28]:
# Get subgraph
subset_nodes = np.unique(df[["Patient ID", "Medication"]].values).tolist()
subG = G.subgraph(subset_nodes).copy()
print(subG)

# Build CREATE command from subgraph
create_command_subG = build_create_command_from_networkx(subG)
print(f"Cypher CREATE command :\n\n{100 * '*'}\n{create_command_subG}\n{100 * '*'}")

DiGraph with 5 nodes and 4 edges
Cypher CREATE command :

****************************************************************************************************
CREATE (n0:Patient {"id":"00009", "displayName":"Christopher BERG", "type":"Patient", "Age":"58", "Date of Admission":"2021-05-23", "Discharge Date":"2021-06-22"}),
(n1:Patient {"id":"00006", "displayName":"Edward EDWARDS", "type":"Patient", "Age":"21", "Date of Admission":"2020-11-03", "Discharge Date":"2020-11-15"}),
(n2:Patient {"id":"00000", "displayName":"Bobby JACKSON", "type":"Patient", "Age":"30", "Date of Admission":"2024-01-31", "Discharge Date":"2024-02-02"}),
(n3:Medication {"id":"Paracetamol", "displayName":"Paracetamol", "type":"Medication"}),
(n4:Patient {"id":"00007", "displayName":"Christina MARTINEZ", "type":"Patient", "Age":"20", "Date of Admission":"2021-12-28", "Discharge Date":"2022-01-07"}),
(n0)-[:TOOK_MEDICATION]-(n3),
(n1)-[:TOOK_MEDICATION]-(n3),
(n2)-[:TOOK_MEDICATION]-(n3),
(n4)-[:TOOK_MEDICATION]-(n3

In [29]:
subgraph_name = f"{graph_name}_subgraph"
subgraph_name

'healthcare_dataset2_subgraph'

In [30]:
%%time

# Create new graph
client.query(f"CREATE GRAPH {subgraph_name}")
client.set_graph(subgraph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW").loc[0, 0]

# Checkout into the change
client.checkout(change=change)

# Run CREATE command
client.query(create_command_subG)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

# Checkout into main
client.checkout()

CPU times: user 3.01 ms, sys: 1.93 ms, total: 4.94 ms
Wall time: 4.66 ms


<div class="alert alert-block alert-info">
    <h2>
        You can visualise the subgraph directly in the notebook below. For more details on nodes and edges, you can go to TuringDB visualizer (running on your instance)
    </h2>
</div>

<div class="alert alert-block alert-info">
    <h2>
        Visualize your graph in TuringDB Graph Visualizer ! Now that your instance is running:
    </h2>
    <h3>
        <ul>
            <li>Go to <a href="https://console.turingdb.ai/databases">TuringDB Console - Database Instances</a></li>
            <li>In your current instance panel, click on "Open Visualizer" button</li>
            <li>Visualizer opens, now you can choose your graph in the dropdown menu at the top-right corner</li>
        </ul>
        You can then play with your graph and visualize the nodes you want !
    </h3>
</div>

In [31]:
from pyvis.network import Network

net = Network(
    height="750px", width="100%", notebook=True, bgcolor="#f8f9fa", font_color="#212529", directed=True
)

# Node type colors
type_colors = {"Patient": "#3498db", "Medication": "#e74c3c"}

for node, data in subG.nodes(data=True):
    node_type = data.get("type", "Unknown")
    color = type_colors.get(node_type, "#7f8c8d")

    label = data.get("displayName", str(node))

    # Build title based on node type
    if node_type == "Patient":
        title = f"<b>{label}</b><br>Age: {data.get('Age', 'N/A')}<br>Admitted: {data.get('Date of Admission', 'N/A')}<br>Discharged: {data.get('Discharge Date', 'N/A')}"
    else:
        title = f"<b>{label}</b><br>Type: {node_type}"

    net.add_node(node, label=label, color=color, title=title, size=25)

# Edge colors by type
edge_colors = {"took_medication": "#27ae60"}

for source, target, data in subG.edges(data=True):
    edge_type = data.get("type", "")
    color = edge_colors.get(edge_type, "#95a5a6")
    net.add_edge(source, target, title=edge_type, color=color, width=3)

net.toggle_physics(True)
net.show(f"{example_name}_subgraph.html")

healthcare_dataset_subgraph.html


# Use LLM to generate Cypher query

Before running this section, create a `.env` file in the project root with your API keys:

```env
ANTHROPIC_API_KEY=your_key_here
OPENAI_API_KEY=your_key_here
MISTRAL_API_KEY=your_key_here

In [32]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [33]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

In [34]:
"""Build system prompt with TuringDB schema and examples"""

turingdb_cypher_system_prompt = """
You are an expert at converting natural language questions into TuringDB queries.

Your task is to generate syntactically correct TuringDB queries based on natural language input.

VERY IMPORTANT - TuringDB Syntax Guidelines:
1. Return ONLY the TuringDB query, no explanations or markdown formatting
2. Use MATCH or CREATE operations only
3. Nodes: (n:Label{property="value"}) or (n:Label{property:value})
4. Edges: Use UNDIRECTED syntax with - (NOT ->)
5. Pattern matching: MATCH (n)-[e]-(m)
6. Property matching: Use = or : operators for exact matching
7. String approximation: Use ~= for approximate string matching
8. Node ID injection: Use @ operator or AT keyword: (n @ 1) or (n AT 1)
9. Multiple constraints: (n:Person,Engineer{name="John", age=30})
10. Return all matched entities: RETURN n, e, m or use RETURN * for all

VERY IMPORTANT - FORBIDDEN in TuringDB:
- Do NOT use directed edges (-> or <-)
- Do NOT use AS aliases
- Do NOT use LIMIT, SKIP clauses
- Do NOT use WHERE clauses
- Do NOT use WITH clauses
- Do NOT use CALL (except for metaqueries)
- Do NOT use toLower() or other functions

Supported TuringDB Operations:
- MATCH queries: MATCH (n:Label)-[e:Type]-(m) RETURN n, m
- CREATE queries: CREATE (n:Label{property="value"})-[e:Type]-(m:Label)
- Metaqueries: CALL PROPERTIES(), CALL LABELS(), CALL EDGETYPES(), CALL LABELSETS()
- Property types: String ("text" or `text`), Boolean (true/false), Integer (20), Unsigned (20u), Double (20.5)

Examples for few-shot learning:
- Find all persons: MATCH (n:Person) RETURN n
- Find connections: MATCH (n:Person)-[e]-(m:Person) RETURN n, e, m
- Create person: CREATE (n:Person{name="John", age=30})
- String approximation: MATCH (n{name~="John"}) RETURN n
- Node by ID: MATCH (n @ 1) RETURN n
- Multiple IDs: MATCH (n:Person @ 1, 2, 3) RETURN n
- Path with 1 hop between Station Paddington and Blackfriars:  MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]-(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, end, end.displayName, end.Note
- Path with 2 hops between Station Paddington and Blackfriars: MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]-(s1:Station)-[e2:CONNECTED]-(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, end, end.displayName, end.Note
- Path with 8 hops between Station Paddington and Blackfriars: MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]-(s1:Station)-[e2:CONNECTED]-(s2:Station)-[e3:CONNECTED]-(s3:Station)-[e4:CONNECTED]-(s4:Station)-[e5:CONNECTED]-(s5:Station)-[e6:CONNECTED]-(s6:Station)-[e7:CONNECTED]-(s7:Station)-[e8:CONNECTED]-(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, s2, s2.displayName, s2.Note, e3.Line, s3, s3.displayName, s3.Note, e4.Line, s4, s4.displayName, s4.Note, e5.Line, s5, s5.displayName, s5.Note, e6.Line, s6, s6.displayName, s6.Note, e7.Line, s7, s7.displayName, s7.Note, e8.Line, end, end.displayName, end.Note
- Find all Chinese providers and what they supply: MATCH (n{provider_country:"CHN"}) RETURN n, n.provider_name, n.displayName, n.share_provided, n.type
- Find all deposition tools and their types: MATCH (specific)-[e:IS_TYPE_OF]-(general:Tool_Resource{displayName:"Deposition tools"}) RETURN specific, specific.displayName, specific.provider_name, e, general, general.displayName
"""

In [35]:
# Get subset of CREATE command to avoid exceeding context window
create_command_subset = create_command.split("\n")[:5] + create_command.split("\n")[-5:]

# Create system_prompt
system_prompt = f"""
TuringDB Cypher prompt :
{turingdb_cypher_system_prompt}

Here is a subset of the CREATE command used to create the graph, this way you know graph structure.
Only a subset is passed because the whole command is to long :
{create_command_subset}

Here is also the output of "CALL LABELS ()" command, showing the different node types of the graph :
{client.query("CALL LABELS ()")}

Here is also the output of "CALL EDGETYPES ()" command, showing the different edge types of the graph :
{client.query("CALL EDGETYPES ()")}

Very important :
- You MUST follow current TuringDB Syntax Guidelines
- You MUST NOT USE what is FORBIDDEN in TuringDB
- By default, RETURN ALL THE MATCHED NODES AND EDGES AND THEIR PROPERTIES in the RETURN section (except contrary demand from user)
- Use the correct node and edge properties name in the MATCH section.
- Use the correct node and edge properties name in the RETURN section.
- Pay attention to which properties come from nodes or edges, to create a functioning query
- Pay attention to lower and uppercases in properties
- If some properties contain spaces, be careful to wrap them

Give me the query FOLLOWING TURINGDB GUIDELINES AND NOT USING WHAT IS FORBIDDEN for this specific question :
"""

In [36]:
# Set natural language query
question = """
Find all patients who took Paracetamol
"""

In [37]:
%%time

provider = "Mistral"

cypher_query = natural_language_to_cypher(
    question=question,
    system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
)
print(f"cypher_query : {cypher_query}")

cypher_query : MATCH (n:Patient)-[e:TOOK_MEDICATION]-(m:Medication{displayName:"Paracetamol"}) RETURN n, e, m
CPU times: user 351 ms, sys: 32 ms, total: 383 ms
Wall time: 870 ms


In [38]:
df_path = client.query(cypher_query)
if df_path.empty:
    print("--> No result found\n")
else:
    df_path.columns = get_return_statements(cypher_query)
    display(df_path)

,n,e,m
0,0,0,4
1,1,1,4
2,2,2,4
3,3,3,4


In [39]:
print("Notebook finished !")

Notebook finished !
